In [ ]:
!pip install pyspark # pyspark is installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
from pyspark import SparkContext, SparkConf #importing packages.

In [ ]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('TF_query of task-1') #spark session is intiated and activated.
sc = SparkContext(conf=conf)

In [ ]:
file2="/content/TF_index_matrix"
fileRDD = sc.textFile(file2) #parallelizing the file into rdd.

In [ ]:
#fileRDD.take(1)

In [ ]:
def conversion_to_rdd(x): #this process is used to convert back to rdd.
  y = x.split('@') #as we have every line wrod@doc-id1#freq1..... splitting on @ to get word in first element of list and remaining in second part.
  z = y[1].split('+') # Again splitting on + as to represent 2 different documents.
  document_dictionary ={} #initialinzing dictionary for storing doc-id as key and freq as value.
  for t in z :
    n = t.split('#') 
    document_dictionary[n[0]] = float(n[1]) #making freq to floats for further processing 

  return (y[0],document_dictionary) #returning a tuple in which first element is word and second element is dictionary of docids as keys and freq as values.

In [ ]:
RDD = fileRDD.map(lambda x : conversion_to_rdd(x)) # rdd has every element of tuples in the mentioned function.

In [ ]:
#RDD.take(1)

In [ ]:
Query = input('Enter your query :- ') #query is asked from user.

Enter your query :- JerusAlem LOVES muslim and hates NA1ZI  in www.google.com


In [ ]:
from tkinter.constants import X
import re
import string #necessary packages are imported.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english')) #stopwords list is created for removal .
def preprocessing(x):
   x = x.lower()  #query words are lowered 
   array1 = x.split(" ") #splitted based on empty space.
   y = ""
   for x1 in array1: 
     if x1 in stop_words: #stopwords are removed if they are in the list of stopwords.
       continue
     else:
       y = y+ " "+ x1 + " "

   urls = re.compile(r'https?://\S+|www\.\S+')
   text_without_urls = urls.sub('', y) #query urls are removed

   text_without_punctuations = re.sub(f"[{re.escape(string.punctuation)}]", " ", text_without_urls) #https://www.geeksforgeeks.org/python-remove-punctuation-from-string/ #query punctuations are removed.

   useful_words = set()
   array2 = text_without_punctuations.split(" ")
   for x1 in array2:
     if len(x1)>0:  #I used set to add filtered words.
       useful_words.add(x1)

   return useful_words #that set is returned.


  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
words  = preprocessing(Query) #query is preprocessed.

In [ ]:
def overal_addition(x):
  return sum(x) #returning the sum of iterators for each key after groupByKey.

In [ ]:
RDD = RDD.filter(lambda x : x[0] in words) #filtering our rdd only when words are in the set.

In [ ]:
#RDD.take(1)

In [ ]:
RDD = RDD.flatMap(lambda x : [(document, x[1][document]) for document in x[1]]) #As we have every element of the form tuple of the form (word,{doc-id1:freq1}...) But our motive is to get documents so iterating in lambda function and taking that key,value pair as tuple and flattening the rdd.
#RDD = RDD.map(lambda x: (x[0], x[1]))
#RDD = RDD.reduceByKey(lambda a, b: a + b)
RDD = RDD.groupByKey() # for same doc-id as key we group all freq values .
RDD = RDD.mapValues(lambda x : overal_addition(x)) #As for common key because of groupbyKey we get a iterator and we will add by passing to mapvalues.

In [ ]:
#RDD.take(4)

[('doc-10000', 1.6989700043360187),
 ('doc-10049', 1.0),
 ('doc-10244', 1.3010299956639813),
 ('doc-10557', 1.6020599913279623)]

In [ ]:
OutputRDD = RDD.sortBy(lambda x: x[1], ascending=False) #from pyspark documentation #using lambda function based on values sorted in descending order.

In [ ]:
OutputRDD = OutputRDD.zipWithIndex()
#To get first 10 elements of RDD without getting out of rdd used zipindex which gives indexing for every elements.

In [ ]:
OutputRDD = OutputRDD.filter(lambda x : x[1]<=9) # taking first 10 document ids based on ids.

In [ ]:
#OutputRDD.count()

In [ ]:
OutputRDD = OutputRDD.map(lambda x : (x[0][0],x[0][1])) #taking as tuples

In [ ]:
#OutputRDD.take(10)
import os
folder_path = "/content/Task1Output1"
OutputRDD.saveAsTextFile(folder_path) #writing back to specifed folder.